In [1]:
import pandas as pd 
import ast 
from tqdm import tqdm 
tqdm.pandas()

In [2]:
import pandas as pd
import torch
from torch.utils.data import DataLoader, TensorDataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [3]:
import os  
import numpy as np 
from tqdm.contrib.concurrent import process_map  
from functools import partial 

import warnings 
warnings.simplefilter('ignore') 

In [5]:
import os 

In [27]:
data = pd.read_csv('../data/all_new.csv')
data 

,Unnamed: 0,datetime,stock_code,title,category,media,news,related,url,month,day,date,weekday
0,0,2023-01-02,NGS,Hoak & Co. Urges NGS Board to Halt CEO Search ...,Investing,Fintel,"Fintel reports that Hoak Public Equities, LP, ...",NGS,https://www.nasdaq.com/articles/hoak-co.-urges...,1,2,2023-01-02,0
1,1,2023-01-02,PFX,Here's Why PhenixFIN (PFX) is Poised for a Tur...,"Stocks,Investing",Zacks,PhenixFIN (PFX) has been beaten down lately wi...,PFX,https://www.nasdaq.com/articles/heres-why-phen...,1,2,2023-01-02,0
2,2,2023-01-02,TYG,My Advice? Sell These 2 Dividend Funds in 2023,"Markets,Stocks",BNK Invest,There's a disconnect setting up in the energy ...,"TYG,TYG,FEN",https://www.nasdaq.com/articles/my-advice-sell...,1,2,2023-01-02,0
3,5,2023-01-02,NVIV,France's InVivo to buy one of the world's olde...,"Commodities,BioTech,US Markets",Reuters,"Malteries Soufflet, a branch of French agribus...",NVIV,https://www.nasdaq.com/articles/frances-invivo...,1,2,2023-01-02,0
4,6,2023-01-02,FEN,My Advice? Sell These 2 Dividend Funds in 2023,"Markets,Stocks",BNK Invest,There's a disconnect setting up in the energy ...,"TYG,TYG,FEN",https://www.nasdaq.com/articles/my-advice-sell...,1,2,2023-01-02,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
146909,625150,2023-08-31,UA,Will Lululemon Stock Trade Higher Post Q2?,"Stocks,US Markets,Investing",Trefis,[Note: Lululemon&rsquo;s FY&rsquo;22 ended on ...,"LULU,LULU,UA,NKE",https://www.nasdaq.com/articles/will-lululemon...,8,31,2023-08-31,3
146910,625282,2023-08-31,Latest,Bain Capital Specialty Finance (BCSF) Price Ta...,Stocks,Fintel,The average one-year price target for Bain Cap...,BCSF,https://www.nasdaq.com/articles/bain-capital-s...,8,31,2023-08-31,3
146911,625283,2023-08-31,Latest,Griffon (GFF) Price Target Increased by 13.60%...,Stocks,Fintel,The average one-year price target for Griffon ...,GFF,https://www.nasdaq.com/articles/griffon-gff-pr...,8,31,2023-08-31,3
146912,625346,2023-08-31,ESE,Esco Technologies (ESE) Price Target Increased...,Stocks,Fintel,The average one-year price target for Esco Tec...,ESE,https://www.nasdaq.com/articles/esco-technolog...,8,31,2023-08-31,3


In [28]:
data.columns

Index(['Unnamed: 0', 'datetime', 'stock_code', 'title', 'category', 'media',
       'news', 'related', 'url', 'month', 'day', 'date', 'weekday'],
      dtype='object')

In [29]:
data = data[['datetime', 'stock_code', 'title', 'category', 'media','news', 'related', 'url', 'month', 'day', 'date', 'weekday']]

# aspect 할당 

In [32]:
topics = {
    0: "Analyst Update",
    1: "Fed | Central Banks",
    2: "Company | Product News",
    3: "Treasuries | Corporate Debt",
    4: "Dividend",
    5: "Earnings",
    6: "Energy | Oil",
    7: "Financials",
    8: "Currencies",
    9: "General News | Opinion",
    10: "Gold | Metals | Materials",
    11: "IPO",
    12: "Legal | Regulation",
    13: "M&A | Investments",
    14: "Macro",
    15: "Markets",
    16: "Politics",
    17: "Personnel Change",
    18: "Stock Commentary",
    19: "Stock Movement"
} 

In [30]:
# GPU 디바이스 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 
device 

device(type='cuda')

In [34]:
model_name = "ProsusAI/finbert"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model_save_path = "../model_ckpt/finbert_finetuned_model_tweet_topic_231113.pth"


# 초기 모델로드 
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(topics), ignore_mismatched_sizes=True)

# 마지막 분류 계층만 초기화
model.classifier.weight.data.normal_(mean=0.0, std=model.config.initializer_range)
model.classifier.bias.data.zero_()

# 학습 정보 불러오기 
model.load_state_dict(torch.load(model_save_path, map_location=torch.device(device)))
model.to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ProsusAI/finbert and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([3, 768]) in the checkpoint and torch.Size([20, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([3]) in the checkpoint and torch.Size([20]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [37]:
def apply_model(sentence, model, tokenizer, mapping_dict = topics): 
    inputs = tokenizer(sentence[:128], return_tensors='pt').to(device)
    with torch.no_grad(): 
        outputs = model(**inputs) 
    prediction = torch.argmax(outputs.logits, dim=1).cpu().item()
    prediction = mapping_dict[prediction] 
    return prediction

In [38]:
data['aspect'] = data['news'].progress_apply(lambda x: apply_model(x, model, tokenizer))

100%|██████████████████████████████████████████████████████████████████████████| 146914/146914 [30:56<00:00, 79.13it/s]


# sentiment 할당 

In [39]:
sentiments = {
    0: "Bearish", 
    1: "Bullish", 
    2: "Neutral"
}  

In [40]:
model_name = "ProsusAI/finbert"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model_save_path = "../model_ckpt/finbert_finetuned_model_tweet_sentiment_231114_oversample6000.pth"

# 초기 모델로드 
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(sentiments), ignore_mismatched_sizes=True)

# 학습 정보 불러오기 
model.load_state_dict(torch.load(model_save_path, map_location=torch.device(device)))
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [41]:
def apply_model(sentence, model, tokenizer, mapping_dict = sentiments): 
    inputs = tokenizer(sentence[:128], return_tensors='pt').to(device)
    with torch.no_grad(): 
        outputs = model(**inputs) 
    prediction = torch.argmax(outputs.logits, dim=1).cpu().item()
    prediction = mapping_dict[prediction] 
    return prediction

In [42]:
data['sentiment'] = data['news'].progress_apply(lambda x: apply_model(x, model, tokenizer))

100%|██████████████████████████████████████████████████████████████████████████| 146914/146914 [32:19<00:00, 75.74it/s]


In [43]:
data 

,datetime,stock_code,title,category,media,news,related,url,month,day,date,weekday,aspect,sentiment
0,2023-01-02,NGS,Hoak & Co. Urges NGS Board to Halt CEO Search ...,Investing,Fintel,"Fintel reports that Hoak Public Equities, LP, ...",NGS,https://www.nasdaq.com/articles/hoak-co.-urges...,1,2,2023-01-02,0,Legal | Regulation,Neutral
1,2023-01-02,PFX,Here's Why PhenixFIN (PFX) is Poised for a Tur...,"Stocks,Investing",Zacks,PhenixFIN (PFX) has been beaten down lately wi...,PFX,https://www.nasdaq.com/articles/heres-why-phen...,1,2,2023-01-02,0,Stock Commentary,Bearish
2,2023-01-02,TYG,My Advice? Sell These 2 Dividend Funds in 2023,"Markets,Stocks",BNK Invest,There's a disconnect setting up in the energy ...,"TYG,TYG,FEN",https://www.nasdaq.com/articles/my-advice-sell...,1,2,2023-01-02,0,Energy | Oil,Neutral
3,2023-01-02,NVIV,France's InVivo to buy one of the world's olde...,"Commodities,BioTech,US Markets",Reuters,"Malteries Soufflet, a branch of French agribus...",NVIV,https://www.nasdaq.com/articles/frances-invivo...,1,2,2023-01-02,0,M&A | Investments,Neutral
4,2023-01-02,FEN,My Advice? Sell These 2 Dividend Funds in 2023,"Markets,Stocks",BNK Invest,There's a disconnect setting up in the energy ...,"TYG,TYG,FEN",https://www.nasdaq.com/articles/my-advice-sell...,1,2,2023-01-02,0,Energy | Oil,Neutral
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146909,2023-08-31,UA,Will Lululemon Stock Trade Higher Post Q2?,"Stocks,US Markets,Investing",Trefis,[Note: Lululemon&rsquo;s FY&rsquo;22 ended on ...,"LULU,LULU,UA,NKE",https://www.nasdaq.com/articles/will-lululemon...,8,31,2023-08-31,3,Company | Product News,Neutral
146910,2023-08-31,Latest,Bain Capital Specialty Finance (BCSF) Price Ta...,Stocks,Fintel,The average one-year price target for Bain Cap...,BCSF,https://www.nasdaq.com/articles/bain-capital-s...,8,31,2023-08-31,3,Analyst Update,Bullish
146911,2023-08-31,Latest,Griffon (GFF) Price Target Increased by 13.60%...,Stocks,Fintel,The average one-year price target for Griffon ...,GFF,https://www.nasdaq.com/articles/griffon-gff-pr...,8,31,2023-08-31,3,Financials,Bullish
146912,2023-08-31,ESE,Esco Technologies (ESE) Price Target Increased...,Stocks,Fintel,The average one-year price target for Esco Tec...,ESE,https://www.nasdaq.com/articles/esco-technolog...,8,31,2023-08-31,3,Financials,Bullish


In [45]:
# data['news'][146910]

In [46]:
data.to_csv('../data/aspect_sentiment_적용1115.csv', index=False, encoding='cp949')